In [1]:
import pandas as pd
import numpy as np
from matplotlib import  pyplot as plt
%matplotlib inline
from time import time


data = pd.read_csv("../data/raw/data.csv",encoding="latin-1")

In [2]:
X = data
X.loc[:, 'materials'] = X.materials.apply(lambda x: str(x).lower().replace('\r', '').replace('\n', ' '))

In [16]:
int(len(X.materials.unique()) * 0.01)

145

In [17]:
list(X.materials.value_counts()[:int(len(X.materials.unique()) * 0.01)].index)

['works on paper',
 'sculpture',
 'oil on canvas',
 'prints',
 'photographs',
 'nan',
 'acrylic on canvas',
 'oil on board',
 'bronze',
 'oil on panel',
 'watercolor',
 'acrylic',
 'color lithograph',
 'pencil',
 'oil on paper',
 'lithograph',
 'gouache',
 'etching',
 'oil on masonite',
 'ink',
 'ink on paper',
 'bronze with brown patina',
 'charcoal',
 'acrylic on paper',
 'chromogenic print',
 'gelatin silver print',
 'pencil and watercolor',
 'c-print',
 'pastel',
 'cibachrome print',
 'oil on linen',
 'watercolor on paper',
 'oilstick on paper',
 'mixed media',
 'gouache on paper',
 'oil on paper on canvas',
 'tempera on paper',
 'color screenprint',
 'offset in colours',
 'screenprint',
 'lithograph in colours',
 'oil on paper laid on canvas',
 'pen and ink',
 'silkscreen in colours',
 'oil and enamel on canvas',
 'dry pigment',
 'oil on burlap',
 'lithograph printed in colours',
 'etching and aquatint',
 'offset lithograph',
 'acrylic and graphite',
 'charcoal on paper',
 'graphi

In [2]:
# drop the duplicates
data = data.drop_duplicates()

# drop instances where hammer_price is NaN or smaller than zero and there is no estimte of high or low
data = data.loc[~np.logical_and(np.logical_and(np.logical_or(data.hammer_price.isnull(),data.hammer_price < 0),data.estimate_high.isnull()),data.estimate_low.isnull())]

# Change Auction_date to datetime objects
data.loc[:,'auction_date'] = pd.to_datetime(data.auction_date)

# Making sure strings are all strings !
data.loc[:,'category'] = data.category.apply(lambda x : str(x).lower().replace('\r','').replace('\n',' '))

data.loc[:,'materials'] = data.materials.apply(lambda x : str(x).lower().replace('\r','').replace('\n',' '))

data.loc[:,'location'] = data.location.apply(lambda x : str(x).lower().replace('\r','').replace('\n',' '))

data.loc[:,'artist_nationality'] = data.artist_nationality.apply(lambda x : str(x).lower().replace('\r','').replace('\n',' '))

In [3]:
replacement_neg_hammer_price_ind = np.logical_and(np.logical_or(data.hammer_price < 0,data.hammer_price.isnull()),np.logical_not(data.estimate_high.isnull()))
replacement_neg_hammer_price = data.loc[replacement_neg_hammer_price_ind,['estimate_high','estimate_low']].mean(axis=1)
data.loc[replacement_neg_hammer_price_ind,'hammer_price'] = replacement_neg_hammer_price
data = data.drop(['estimate_high','estimate_low'],axis=1)


In [4]:
GBP_USD = 1.41
EUR_USD = 1.25
data.loc[data.currency=='GBP','hammer_price'] = data.loc[data.currency=='GBP','hammer_price'].apply(lambda x : x*GBP_USD)
data.loc[data.currency=='EUR','hammer_price'] = data.loc[data.currency=='EUR','hammer_price'].apply(lambda x : x*EUR_USD)
# Dropping the Currency type after cleaning
data = data.drop('currency',axis=1)

In [5]:
data.loc[data.category=="other works on paper","category"] = "painting"
data.loc[np.logical_and(data.materials=="oil on canvas",data.category=="unclassified"),'category']="painting"

data.loc[np.logical_and(data.materials=="works on paper",data.category=="unclassified"),"category"]="painting"

data.loc[np.logical_and(data.materials=="oil and charcoal",data.category=="unclassified"),"category"]="painting"

data.loc[np.logical_and(data.materials=="sculpture",data.category=="unclassified"),"category"]="sculpture"

data = data.drop(data.loc[np.logical_and(data.category=="unclassified",data.materials.isnull())].index)

In [18]:
## Create Materials
valid_materials = list(data.materials.value_counts()[data.materials.value_counts() > 5000].index)

# The only valid materials are paper,canvas,prints,sculpture and other
valid_materials = ['paper','sculpture','canvas','prints']
def clean_materials(x):
    for m in valid_materials:
        if m in x.materials:
            return m
    return 'other'

data = data.assign(material = data.apply(clean_materials,axis=1))
data = data.drop('materials',axis=1)

In [7]:
data = data.drop(['title'],axis=1)
data = data.drop_duplicates()

In [8]:
def clean_location(x):        
    x = str(x).lower()
    if "," in x:
        return x.split(',')[-1].strip(" ")
    return x.strip(" ")
data.loc[:,'location']=data.location.apply(clean_location)
valid_locs = list(data.location.value_counts()[data.location.value_counts() > 100].index)
data.loc[~data.location.isin(valid_locs),"location"] = "other"
data.loc[data.location=="nan","location"] = "other"

In [9]:
data.loc[:,'auction_date'] = pd.to_datetime(data.auction_date)
data = data.assign(year =[x.year for x in data.auction_date], month=[x.month for x in data.auction_date],day=[x.day for x in data.auction_date],week=[x.week for x in data.auction_date])
data = data.drop('auction_date',axis=1)

In [10]:
def surface_volume(x):
    if x.measurement_depth_cm == 0:
        return x.measurement_width_cm * x.measurement_height_cm
    else:
        return x.measurement_width_cm * x.measurement_height_cm * x.measurement_depth_cm
data = data.assign(surface = data.loc[:,['measurement_width_cm','measurement_height_cm','measurement_depth_cm']].apply(surface_volume,axis=1))


In [11]:
data = data.assign(is_artist_dead = (~data.artist_death_year.isnull()).astype(np.int))

In [12]:
data = data.drop('year_of_execution',axis=1)

In [13]:
## add aspect ratio
def calc_aspect_ratio(x):
    if x.measurement_width_cm > 0 and x.measurement_height_cm > 0:
        return x.measurement_width_cm/x.measurement_height_cm
    return 0

data = data.assign(aspect_ratio = data.loc[:,["measurement_width_cm","measurement_height_cm"]].apply(calc_aspect_ratio,axis=1))

In [14]:
# how long ago this got sold
current_year = 2018
data = data.assign(years_sold = current_year - data.year)


In [15]:
# Number of years since the artist was born
data = data.assign(years_sold = current_year - data.artist_birth_year)

In [16]:
dropped_attr = ['artist_birth_year',
                'artist_death_year',
                'edition',
               ]

data = data.drop(dropped_attr,axis=1)

In [22]:
data = data.loc[data.hammer_price > 0]

In [27]:
data = data.drop_duplicates()

In [28]:
data.to_csv('../data/interim/003-raw-clean-data.csv',encoding="utf-8")